一、线性回归模型

In [1]:
# 1. 准备数据

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch import nn

# sample number
n = 400

# 生成测试数据集
X = 10 * torch.rand([n, 2]) - 5.0  # torch.randn 是均匀分布类型
w0 = torch.tensor([[2.0], [-3.0]])
b0 = torch.tensor([[10.0]])

y = X @ w0 + b0 + torch.normal(0.0, 2.0, size=[n, 1])


In [ ]:


plt.figure(figsize=(12, 5))
ax1 = plt.subplot(121)
ax1.scatter(X[:, 0].numpy(), y[:, 0].numpy(), c='b', label='samples')
ax1.legend()
plt.xlabel('x1')
plt.ylabel('y', rotation=0)

ax2= plt.subplot(122)
ax2.scatter(X[:, 1].numpy(), y[:, 0].numpy(), c='g', label='samples')
ax2.legend()
plt.xlabel('x2')
plt.ylabel('y', rotation=0)
plt.show()

In [3]:
# 构建数据管道迭代器
def data_iter(features, labels, batch_size=8):
    num_examples = len(features)
    indices = list(range(num_examples))
    np.random.shuffle(indices)  #样本的读取顺序是随机的
    for i in range(0, num_examples, batch_size):
        indexs = torch.LongTensor(indices[i: min(i + batch_size, num_examples)])
        yield  features.index_select(0, indexs), labels.index_select(0, indexs)
        
# 测试数据管道效果   
batch_size = 8
(features,labels) = next(data_iter(X, y,batch_size))
print(features)
print(labels)

tensor([[ 0.1990, -4.5614],
        [-0.8613, -1.3396],
        [-1.1601,  2.1362],
        [-2.5265, -0.8018],
        [ 2.1353, -0.2573],
        [-0.4110, -2.0204],
        [-1.9846,  4.0192],
        [-1.8882,  0.1019]])
tensor([[27.1468],
        [12.6352],
        [-1.3083],
        [ 6.9933],
        [14.7112],
        [12.8322],
        [-5.2764],
        [ 5.9865]])


2. 定义模型

In [4]:
class LinearRegression:

    def __init__(self):
        self.w = torch.randn_like(w0, requires_grad=True)
        self.b = torch.zeros_like(b0, requires_grad=True)

    # 正向传播
    def forward(self, x):
        return x@self.w + self.b

    # 损失函数
    def loss_func(self, y_pred, y_true):
        return torch.mean((y_pred-y_true)**2 / 2)

model = LinearRegression()

3. 训练模型

In [7]:
def train_step(model, features, labels):

    predictions = model.forward(features)
    loss = model.loss_func(predictions, labels)

    # 反向传播求梯度
    loss.backward()

    # 使用torch.no_grad() 避免梯度记录， 也可以通过操作model.w.data 实现避免梯度记录
    with torch.no_grad():
        # 梯度下降法更新参数
        model.w -= 0.01 * model.w.grad
        model.b -= 0.01 * model.b.grad

        # 梯度清零
        model.w.grad.zero_()
        model.b.grad.zero_()

    return loss

In [8]:
# 测试train_step 效果
batch_size = 10
features, labels = next(data_iter(X, y, batch_size))
train_step(model, features, labels)

tensor(41.0938, grad_fn=<MeanBackward0>)

In [10]:
def train_model(model, epochs):
    for epoch in range(1, epochs+1):
        for features, labels in data_iter(X, y, 10):
            loss = train_step(model, features, labels)
        
        if epoch % 20 == 0:
            print('epoch =', epoch, 'loss=', loss.item())
            print('model.w =', model.w.data)
            print('model.b =', model.b.data)

train_model(model, epochs=1000)

epoch = 20 loss= 1.8846652507781982
model.w = tensor([[ 1.9296],
        [-2.9726]])
model.b = tensor([[10.0611]])
epoch = 40 loss= 2.7229256629943848
model.w = tensor([[ 1.8898],
        [-3.0089]])
model.b = tensor([[10.0828]])
epoch = 60 loss= 1.5944843292236328
model.w = tensor([[ 1.9580],
        [-3.0137]])
model.b = tensor([[10.0838]])
epoch = 80 loss= 2.19413161277771
model.w = tensor([[ 1.9293],
        [-2.9985]])
model.b = tensor([[10.0710]])
epoch = 100 loss= 2.581402063369751
model.w = tensor([[ 1.9435],
        [-3.0479]])
model.b = tensor([[10.0680]])
epoch = 120 loss= 1.622877836227417
model.w = tensor([[ 1.9781],
        [-3.0047]])
model.b = tensor([[10.0785]])
epoch = 140 loss= 5.3971638679504395
model.w = tensor([[ 1.9310],
        [-3.0045]])
model.b = tensor([[10.0787]])
epoch = 160 loss= 2.0983893871307373
model.w = tensor([[ 1.9302],
        [-3.0364]])
model.b = tensor([[10.0666]])
epoch = 180 loss= 1.9165757894515991
model.w = tensor([[ 1.9326],
        [-3.04

In [11]:
# 结果可视化

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

plt.figure(figsize = (12,5))
ax1 = plt.subplot(121)
ax1.scatter(X[:,0].numpy(),Y[:,0].numpy(), c = "b",label = "samples")
ax1.plot(X[:,0].numpy(),(model.w[0].data*X[:,0]+model.b[0].data).numpy(),"-r",linewidth = 5.0,label = "model")
ax1.legend()
plt.xlabel("x1")
plt.ylabel("y",rotation = 0)


ax2 = plt.subplot(122)
ax2.scatter(X[:,1].numpy(),Y[:,0].numpy(), c = "g",label = "samples")
ax2.plot(X[:,1].numpy(),(model.w[1].data*X[:,1]+model.b[0].data).numpy(),"-r",linewidth = 5.0,label = "model")
ax2.legend()
plt.xlabel("x2")
plt.ylabel("y",rotation = 0)

plt.show()

二、 DNN 二分类模型